In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
import glob

In [2]:
results_folder = "results"
df_experiment = pd.DataFrame()

In [3]:
for experiment_folder in os.listdir(results_folder):
    experiment_files = os.listdir(os.path.join(results_folder,experiment_folder))

    if len(experiment_files) != 3:
        continue
    
    for file in experiment_files:
        
        read_file = os.path.join(results_folder,experiment_folder,file)

        
        
        if "config" in file:
            df_config =pd.read_csv(read_file)
            
        if "test_metrics" in file:
            df_test = pd.read_csv(read_file)
            
        if "benchmark" in file:
            df_benchmark = pd.read_csv(read_file)
            
            avg_time = df_benchmark.Time.mean()
            avg_pop = df_benchmark.Avg.iloc[-1]
            best_pop = df_benchmark.Best.iloc[-1]
            worst_pop = df_benchmark.Worst.iloc[-1]
            
            df_benchmark = pd.DataFrame({"Time": [avg_time],"Avg":[avg_pop],"Worst":[worst_pop],"Best":[best_pop]})

    row = pd.concat((df_test,df_benchmark,df_config),axis=1)
    df_experiment = pd.concat((df_experiment,row))


In [8]:
df_experiment.columns

Index(['F1', 'Accuracy', 'ROC AUC', 'Best Chromosome', 'Time', 'Avg', 'Worst',
       'Best', 'dataset', 'crossover_choice', 'mutation_rate', 'metric_choice',
       'population_size', 'elitism', 'evolution_rounds', 'stopping_threshold',
       'model', 'algorithm', 'backend_prefer'],
      dtype='object')

In [4]:
models = ["mlp","xgboost","logistic"]
datasets = ["gina_agnostic","hiva_agnostic","sylva_agnostic"]
algorithms = ["rfs","ga_seq","random","ga_joblib"][::-1]

In [5]:
df_experiment.groupby(["dataset","model"])[["F1","Accuracy","ROC AUC"]].mean().reindex(index=models, level=1)

F1  Accuracy   ROC AUC
dataset        model                                 
gina_agnostic  mlp       0.886215  0.890490  0.890135
               xgboost   0.923305  0.925072  0.924924
               logistic  0.800014  0.803074  0.803019
hiva_agnostic  mlp       0.421818  0.971631  0.644951
               xgboost   0.352982  0.967849  0.623725
               logistic  0.347003  0.959023  0.646977
sylva_agnostic mlp       0.934361  0.991898  0.964195
               xgboost   0.937853  0.992361  0.964442
               logistic  0.938072  0.992460  0.961496

In [59]:
df_experiment[df_experiment.backend_prefer=="processes"].reset_index(drop=True).groupby(["dataset","model","algorithm"])[["F1","Accuracy","ROC AUC"]].mean().unstack(level="dataset").reindex(index=algorithms, level=1).reindex(index=models,level=0).stack(0).unstack()#.swaplevel(0, 1).sort_index(axis=1, level=0)

dataset            gina_agnostic                     hiva_agnostic            \
                        Accuracy        F1   ROC AUC      Accuracy        F1   
model    algorithm                                                             
mlp      ga_joblib      0.896254  0.892216  0.895900      0.971631  0.400000   
         random         0.881844  0.876877  0.881446      0.971631  0.454545   
         ga_seq         0.896254  0.892216  0.895900      0.971631  0.400000   
xgboost  ga_joblib      0.919308  0.918605  0.919375      0.966903  0.416667   
         random         0.930836  0.928571  0.930572      0.966903  0.300000   
         ga_seq         0.919308  0.918605  0.919375      0.966903  0.416667   
logistic ga_joblib      0.806916  0.803519  0.806835      0.959811  0.320000   
         random         0.795389  0.793003  0.795388      0.955083  0.387097   
         ga_seq         0.806916  0.803519  0.806835      0.959811  0.320000   
         rfs                 NaN       NaN       NaN           NaN       NaN   

dataset                      sylva_agnostic                      
                     ROC AUC       Accuracy        F1   ROC AUC  
model    algorithm                                               
mlp      ga_joblib  0.632108       0.991667  0.933333  0.969319  
         random     0.664216       0.992361  0.936416  0.953946  
         ga_seq     0.632108       0.991667  0.933333  0.969319  
xgboost  ga_joblib  0.661765       0.992361  0.937853  0.964442  
         random     0.597549       0.992361  0.937853  0.964442  
         ga_seq     0.661765       0.992361  0.937853  0.964442  
logistic ga_joblib  0.625980       0.993750  0.948571  0.965182  
         random     0.687745       0.992361  0.937853  0.964442  
         ga_seq     0.625980       0.993750  0.948571  0.965182  
         rfs             NaN       0.987500  0.895349  0.930364

In [58]:
df_experiment[df_experiment.backend_prefer=="processes"].reset_index(drop=True).groupby(["dataset","model","algorithm"])[["Time"]].mean().unstack(level="dataset").reindex(index=algorithms,level=1).reindex(index=models,level=0)

Time                             
dataset            gina_agnostic hiva_agnostic sylva_agnostic
model    algorithm                                           
mlp      ga_joblib      4.620493     14.108735       7.403592
         random        87.422665    196.429952     219.697947
         ga_seq        87.653716    199.970677     235.539700
xgboost  ga_joblib     10.106024     14.164435      11.554285
         random        33.139521     26.135362      23.340337
         ga_seq        16.298722     76.499615      21.485557
logistic ga_joblib      1.101781      2.087491       1.819287
         random         2.886472      3.589068       5.513100
         ga_seq         3.218098      4.496395       8.970394
         rfs                 NaN           NaN   11938.580672

In [35]:
import itertools

In [38]:
list(itertools.product(algorithms,algorithms))

[('ga_joblib', 'ga_joblib'),
 ('ga_joblib', 'random'),
 ('ga_joblib', 'ga_seq'),
 ('ga_joblib', 'rfs'),
 ('random', 'ga_joblib'),
 ('random', 'random'),
 ('random', 'ga_seq'),
 ('random', 'rfs'),
 ('ga_seq', 'ga_joblib'),
 ('ga_seq', 'random'),
 ('ga_seq', 'ga_seq'),
 ('ga_seq', 'rfs'),
 ('rfs', 'ga_joblib'),
 ('rfs', 'random'),
 ('rfs', 'ga_seq'),
 ('rfs', 'rfs')]

In [57]:
df_subset["Best Chromosome"]

0    [0 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0...
1    [1 1 1 0 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 1 1...
2    [0 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0...
3                                    [0 1 1 ... 1 0 1]
4                                    [0 1 1 ... 1 1 0]
5    [1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1...
6    [1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1...
7                                    [0 1 1 ... 1 1 0]
8    [1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 1 0 0 1 1 1 1 0...
Name: Best Chromosome, dtype: object

In [70]:
from scipy.spatial.distance import jaccard

In [83]:
def feature_jaccard_overlap(df,dataset,algorithms):
    pairwise_comparisons = list(itertools.product(algorithms,algorithms))
    df = df[df.dataset==dataset]
    for algo1,algo2 in pairwise_comparisons:
        try:
            chromosome1 = eval(df[df.algorithm==algo1]["Best Chromosome"].values[0].replace(" ",","))
            chromosome2 = eval(df[df.algorithm==algo2]["Best Chromosome"].values[0].replace(" ",","))
            # print(chromosome1,chromosome2)
            print(algo1,algo2,1-jaccard(chromosome1,chromosome2))
        except:
            continue

df_subset = df_experiment[(df_experiment.backend_prefer=="processes") & (df_experiment.model=="logistic")].reset_index(drop=True)
feature_jaccard_overlap(df=df_subset,dataset="sylva_agnostic",algorithms=algorithms)

ga_joblib ga_joblib 1.0
ga_joblib random 0.29824561403508776
ga_joblib ga_seq 1.0
ga_joblib rfs 0.3167259786476868
random ga_joblib 0.29824561403508776
random random 1.0
random ga_seq 0.29824561403508776
random rfs 0.3807829181494662
ga_seq ga_joblib 1.0
ga_seq random 0.29824561403508776
ga_seq ga_seq 1.0
ga_seq rfs 0.3167259786476868
rfs ga_joblib 0.3167259786476868
rfs random 0.3807829181494662
rfs ga_seq 0.3167259786476868
rfs rfs 1.0


In [46]:
df_experiment[(df_experiment.backend_prefer=="processes") & (df_experiment.model=="mlp")]

,F1,Accuracy,ROC AUC,Best Chromosome,Time,Avg,Worst,Best,dataset,crossover_choice,mutation_rate,metric_choice,population_size,elitism,evolution_rounds,stopping_threshold,model,algorithm,backend_prefer
0,0.892216,0.896254,0.895900,[0 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0...,4.620493,0.922625,0.897898,0.938053,gina_agnostic,onepoint,0.2,f1,50,2,50,0.99,mlp,ga_joblib,processes
0,0.876877,0.881844,0.881446,[1 1 1 0 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 1 1...,87.422665,0.908377,0.893233,0.924444,gina_agnostic,onepoint,0.2,f1,50,2,50,0.99,mlp,random,processes
0,0.892216,0.896254,0.895900,[0 1 0 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 0 0...,87.653716,0.922625,0.897898,0.938053,gina_agnostic,onepoint,0.2,f1,50,2,50,0.99,mlp,ga_seq,processes
0,0.454545,0.971631,0.664216,[0 1 1 ... 1 0 1],196.429952,0.396972,0.304348,0.509804,hiva_agnostic,onepoint,0.2,f1,50,2,50,0.99,mlp,random,processes
0,0.400000,0.971631,0.632108,[0 1 1 ... 1 1 0],199.970677,0.472558,0.375000,0.566038,hiva_agnostic,onepoint,0.2,f1,50,2,50,0.99,mlp,ga_seq,processes
0,0.933333,0.991667,0.969319,[1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1...,7.403592,0.954422,0.920455,0.975069,sylva_agnostic,onepoint,0.2,f1,50,2,50,0.99,mlp,ga_joblib,processes
0,0.933333,0.991667,0.969319,[1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1...,235.539700,0.954422,0.920455,0.975069,sylva_agnostic,onepoint,0.2,f1,50,2,50,0.99,mlp,ga_seq,processes
0,0.400000,0.971631,0.632108,[0 1 1 ... 1 1 0],14.108735,0.472558,0.375000,0.566038,hiva_agnostic,onepoint,0.2,f1,50,2,50,0.99,mlp,ga_joblib,processes
0,0.936416,0.992361,0.953946,[1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 1 0 0 1 1 1 1 0...,219.697947,0.904750,0.821839,0.938202,sylva_agnostic,onepoint,0.2,f1,50,2,50,0.99,mlp,random,processes
